# 해시 함수에서의 패딩

### ✅ 왜 패딩을 할까?

해시 함수는 보통 **고정된 블록 크기**(예: 512비트, 1024비트 등)를 입력으로 처리합니다.

그런데 우리가 해싱하려는 **원래 메시지**는 크기가 다양하죠?

그래서 해시 함수는 내부적으로:

- 입력 메시지를 **고정 블록 크기 단위로 맞추기 위해**
- *패딩(Padding)**을 추가합니다.

---

### ✅ 어떻게 패딩하나? (SHA 계열 기준)

예를 들어 **SHA-256**은 **512비트 블록**을 사용하므로 다음 순서로 패딩이 이루어집니다:

1. **원래 메시지 뒤에 1비트 추가**: `100000...` 형태로 시작 (시작 표시)
2. **0비트들을 채워서** 총 메시지 길이가 **[마지막 64비트를 제외하고] 512의 배수 - 64비트**가 되게 함
3. **마지막 64비트는 원래 메시지의 비트 길이**를 나타냄

> 즉:
>
>
> `[원래 메시지] + 1 + 0들 + [메시지 비트 길이]` = **512의 배수**
>

---

### 🧠 예시 (SHA-256)

- 원래 메시지가 1000비트라면?

    → 다음 블록을 채우기 위해 24비트만큼 패딩이 들어가고

    → 마지막 64비트에는 **1000을 64비트로 표현한 이진값**이 붙음.


---

### 💡 패딩은 왜 보안과 연결되나?

- 메시지 길이도 해시 값에 영향을 주기 때문에

    → **중간 메시지 공격 등**을 막기 위한 중요한 방어 요소예요.

- "길이 확장 공격(length extension attack)" 같은 걸 막는 데도 활용돼요 (특히 MAC 설계시 중요).

---

### ✅ 한 줄 요약:

> 해시에서 패딩은 "입력 메시지를 고정 블록 크기로 맞추기 위해, 1비트 + 0 + 메시지 길이"를 추가하는 과정입니다.
>
>

---


In [4]:
def calculate_padding_sha(algorithm: str, message_len_bits: int):
    """
    Calculate padding length and total size for SHA-256 or SHA-512.
    """
    if algorithm == 'sha256':
        block_size = 512
        length_field = 64
    elif algorithm == 'sha512':
        block_size = 1024
        length_field = 128
    else:
        raise ValueError("Only 'sha256' and 'sha512' are supported.")

    # Calculate k: number of 0 bits to pad after 1-bit (0x80)
    k = (block_size - length_field - (message_len_bits + 1)) % block_size
    total_bits = message_len_bits + 1 + k + length_field

    return {
        "algorithm": algorithm.upper(),
        "original_message_length_bits": message_len_bits,
        "block_size": block_size,
        "length_field_bits": length_field,
        "padding_1bit": 1,
        "padding_zeros_bits": k,
        "total_padded_length_bits": total_bits,
        "num_blocks": total_bits // block_size
    }

# Compare SHA-256 and SHA-512 for a 3940-bit message
sha256_result = calculate_padding_sha('sha256', 3940)
sha512_result = calculate_padding_sha('sha512', 3940)

import pandas as pd
df = pd.DataFrame([sha256_result, sha512_result])
import ace_tools_open as tools;  tools.display_dataframe_to_user(name="SHA Padding Comparison", dataframe=df)


SHA Padding Comparison


Loading ITables v2.4.4 from the internet... (need help?)


### ✅ 1. **원래 메시지 비트 (Original Message Bits)**

- 메시지 자체의 **내용**입니다.
- 예:

    `"ABC"` = `0x41 0x42 0x43` → 3바이트 = **24비트**


즉,

> "원래 메시지 비트"는 해시 함수에 넣으려는 입력 데이터 그 자체의 길이
>

---

### ✅ 2. **원래 메시지 길이 비트 (Length Field in Padding)**

- 패딩의 마지막에 추가되는 **고정 크기 필드** (SHA-256은 64비트, SHA-512는 128비트)
- 그 안에 들어가는 값은 바로 **"원래 메시지 비트 수"를 표현한 숫자**예요.

즉,

> **입력 메시지의 길이(L)**를 이진수로 표현해서 마지막에 부착하는 게 이 필드입니다.
>

---

### 🔁 관계 요약

| 항목 | 예시 (`"ABC"`) | 설명 |
| --- | --- | --- |
| 원래 메시지 | `"ABC"` | 실제 데이터 |
| 원래 메시지 비트 수 (L) | 24비트 (3바이트) | `len(message) * 8` |
| 패딩 마지막 필드 | `000...011000` (64비트) | 숫자 `24`를 64비트로 표현한 것 |

---

### 📌 정리 문장:

> "패딩의 마지막 64비트/128비트는, 원래 메시지의 비트 수를 이진수로 표현한 것이다."
>